# <span style="color:hotpink"> PySpark labo </span> - Nathan Segers

BEGIN MET HET INVULLEN VAN JE NAAM

Naam: 

Tijdens dit deel van het labo zal je de PySpark basics leren en hoe ze toe te passen voor machine learning.  
De dataset waarmee gewerkt zal worden is data over huizen in king county en op het einde zal je de prijs van een huis proberen te voorspellen aan de hand van lineaire regressie.  
De kolommen die aanwezig zijn in de dataset en hun beschrijving zijn:  
`price`: prediction target  
`bedrooms`: number of bedrooms/house  
`bathrooms`: number of bathrooms/house  
`sqft_living`: square footage of the home  
`sqft_lot`: square footage of the lot  
`floors`: Total floors(levels) in a house  
`waterfront`: House which has a view to a waterfront  
`view`: Has been viewed  
`condition`: How good the condition overall is  
`grade`: overall grade given to the housing unit, based on King County grading system  
`sqft_above`: square footage of house apart from basement  
`sqft_basement`: square footage of the basement  
`yr_built`: built year  
`yr_renovated`: Year when house was renovated  
`zipcode`: zip  
`sqft_living15`: Living room area in 2015(implies some renovations) This might or might not have affected the lotsize area  
`sqft_lot15`: LotSize area in 2015(implies some renovation) 

De te beantwoorden vragen/opdrachten staan altijd in italics. 

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression

## <span style="color:hotpink"> SparkSession, data inlezen en data weergeven</span>
Een SparkSession voorziet een single point of entry om te interageren met de onderliggende Spark functionaliteit en laat de gebruiker toe om te programmeren met de DataFrame API.  
De te gebruiken functie om een sparksessie te bouwen is:  
`spark = SparkSession.builder.appName('name').getOrCreate()`  
*Bouw nu zelf een SparkSession met de naam labo.*

In [2]:
spark = SparkSession.builder.appName("labo").getOrCreate()

De data kan ingelezen worden met de functie: `spark.read.csv()`  
*Laad de data `kc_house_data.csv` in en zet de parameters `inferSchema` en `header` op True*

In [134]:
data = spark.read.csv("kc_house_data.csv", inferSchema = True, header = True)

De functie `df.printSchema()` toont alle kolommen en het type waarden dat hierin aanwezig zijn.  

In [4]:
data.printSchema()

root
 |-- id: long (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: double (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: double (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)
 |-- condition: integer (nullable = true)
 |-- grade: integer (nullable = true)
 |-- sqft_above: integer (nullable = true)
 |-- sqft_basement: integer (nullable = true)
 |-- yr_built: integer (nullable = true)
 |-- yr_renovated: integer (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- sqft_living15: integer (nullable = true)
 |-- sqft_lot15: integer (nullable = true)



*Wat is het type waarden aanwezig in de kolom sqft_basement?*

een integer

Om een spark DataFrame (of spark column) weer te geven moet je de functie `df.show()` aanroepen. Dus telkens een functie een Spark Dataframe returned moet je `.show()` gebruiken om het resultaat te kunnen zien.  
*Bekijk nu zelf de data.*

In [5]:
data.show()

+----------+-------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+-------------+----------+
|        id|  price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|condition|grade|sqft_above|sqft_basement|yr_built|yr_renovated|zipcode|sqft_living15|sqft_lot15|
+----------+-------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+-------------+----------+
|7129300520| 221900|       3|      1.0|       1180|    5650|   1.0|         0|   0|        3|    7|      1180|            0|    1955|           0|  98178|         1340|      5650|
|6414100192| 538000|       3|     2.25|       2570|    7242|   2.0|         0|   0|        3|    7|      2170|          400|    1951|        1991|  98125|         1690|      7639|
|5631500400| 180000|       2|      1.0|        770|   10000|   1.0|         0|   0|        3|    6| 

Om de namen van de kolom weer te geven als lijst kan je de functie:  
`df.columns` gebruiken

In [6]:
data.columns

['id',
 'price',
 'bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated',
 'zipcode',
 'sqft_living15',
 'sqft_lot15']

# <span style="color:hotpink"> selecteren van kolommen </span>
Je kan een kolom selecteren aan de hand van de methode:  
`df["column_name"]`  
Selecteer de kolom condition

In [7]:
data["condition"]

Column<condition>

Het verkregen type is een Column, maar met de functie `df.select('column_name')` returened een spark DataFrame, die veelzijdiger is dan het type Column.  
*Selecteer opnieuw de kolom condition maar nu als dataframe.*

In [8]:
data.select("condition")

DataFrame[condition: int]

Om de inhoud van de geselecteerde kolom weer te geven moet je de methode `.show` aanroepen.

In [9]:
data.select("condition").show()

+---------+
|condition|
+---------+
|        3|
|        3|
|        3|
|        5|
|        3|
|        3|
|        3|
|        3|
|        3|
|        3|
|        3|
|        4|
|        4|
|        4|
|        3|
|        3|
|        3|
|        4|
|        4|
|        4|
+---------+
only showing top 20 rows



`df.head(n)` geeft de eerste n rijen terug als lijst.  
Print de eerste 5 rijen

In [10]:
data.head(5)

[Row(id=7129300520, price=221900, bedrooms=3, bathrooms=1.0, sqft_living=1180, sqft_lot=5650, floors=1.0, waterfront=0, view=0, condition=3, grade=7, sqft_above=1180, sqft_basement=0, yr_built=1955, yr_renovated=0, zipcode=98178, sqft_living15=1340, sqft_lot15=5650),
 Row(id=6414100192, price=538000, bedrooms=3, bathrooms=2.25, sqft_living=2570, sqft_lot=7242, floors=2.0, waterfront=0, view=0, condition=3, grade=7, sqft_above=2170, sqft_basement=400, yr_built=1951, yr_renovated=1991, zipcode=98125, sqft_living15=1690, sqft_lot15=7639),
 Row(id=5631500400, price=180000, bedrooms=2, bathrooms=1.0, sqft_living=770, sqft_lot=10000, floors=1.0, waterfront=0, view=0, condition=3, grade=6, sqft_above=770, sqft_basement=0, yr_built=1933, yr_renovated=0, zipcode=98028, sqft_living15=2720, sqft_lot15=8062),
 Row(id=2487200875, price=604000, bedrooms=4, bathrooms=3.0, sqft_living=1960, sqft_lot=5000, floors=1.0, waterfront=0, view=0, condition=5, grade=7, sqft_above=1050, sqft_basement=910, yr_bu

Je kan met `data.head()` ook een specifieke rij of een bepaalde waarde selecteren met behulp van:  
`data.head(n)[row_number][column_number]`  
*1) Selecteer de 4de rij*  
*2) Selecteer prijs op de 3de rij*

In [11]:
data.head(20)[3] # Index == 0

Row(id=2487200875, price=604000, bedrooms=4, bathrooms=3.0, sqft_living=1960, sqft_lot=5000, floors=1.0, waterfront=0, view=0, condition=5, grade=7, sqft_above=1050, sqft_basement=910, yr_built=1965, yr_renovated=0, zipcode=98136, sqft_living15=1360, sqft_lot15=5000)

In [12]:
data.head(5)[2][1]

180000

*Is dit een action of transformation?*

Action

# <span style="color:hotpink"> Creating a new column and dropping columns </span>

Je kan een nieuwe kolom aanmaken met de functie: `data.withColumn('new_column_name',data["column"])`  
data["column"] kan ook bewerkt worden bv: `data.withColumn('new_column_name',data["column"]/2)`  
Een kolom kan verwijderd worden met `data.drop('column')`  
_Verander nu alle sqft kolommen naar m<sup>2</sup> (*0.0929) en verwijder de sqft kolommen._

In [139]:
data = data.withColumn("m2_living", data["sqft_living"] * 0.0929)
data = data.drop("sqft_living")

In [140]:
data = data.withColumn("m2_lot", data["sqft_lot"] * 0.0929)
data = data.drop("sqft_lot")

In [141]:
data = data.withColumn("m2_above", data["sqft_above"] * 0.0929)
data = data.drop("sqft_above")

In [142]:
data = data.withColumn("m2_basement", data["sqft_basement"] * 0.0929)
data = data.drop("sqft_basement")

In [143]:
data = data.withColumn("m2_living15", data["sqft_living15"] * 0.0929)
data = data.drop("sqft_living15")

In [144]:
data = data.withColumn("m2_lot15", data["sqft_lot15"] * 0.0929)
data = data.drop("sqft_lot15")

*Is dit een action of transformation?*

transformation

In [19]:
data.columns

['id',
 'price',
 'bedrooms',
 'bathrooms',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'yr_built',
 'yr_renovated',
 'zipcode',
 'm2_living',
 'm2_lot',
 'm2_above',
 'm2_basement',
 'm2_living15',
 'm2_lot15']

# <span style="color:hotpink"> Filtering and grouping data </span>

## <span style="color:hotpink"> Filtering Data </span>

Voor het werken met big data is het belangrijk dat je snel je data kan filteren gebaseerd op bepaalde condities.  

Je kunt de dataset filteren met behulp van de methode:  
`df.filter(df["column_name"] < condition)`  
Je kan ook filteren voor meerdere condities met behulp van `|` of `&`

*Filter de dataset voor huizen die gebouwd zijn na het jaar 2000, sla dit op onder de variabele `data_2000`*

In [20]:
data_2000 = data.filter(data["yr_built"] > 2000)

Met de `.count()` methode kan je het aantal gefilterde huizen opvragen.  
*Hoeveel huizen bevat `dataset_2000` met minimum 2 slaapkamers en die gelegen zijn aan het waterfront?*

In [21]:
data_2000.filter((data["bedrooms"] >= 2) & (data["waterfront"] == 1)).count()

18

## <span style="color:hotpink"> GroupBy and Aggregate Functions </span>

De `groupBy('column_name')` functie laat je rijen groeperen gebaseerd op een bepaalde kolom, bijvoorbeeld je kan huizen groeperen volgens bouwjaar.  
Eenmaal je de rijen gegroepeerd hebt, kan je meerdere rijen van data aggregeren tot een output, bijvoorbeeld door het nemen van de som van alle inputrijen of de minimum waarde.  
*Gebruik de groupBy functie op de `condition` kolom en vraag de minimum waarde op met de `.min()` functie.*

In [22]:
data.groupBy("condition").min().show()

+---------+--------+----------+-------------+--------------+-----------+---------------+---------+--------------+----------+-------------+-----------------+------------+--------------+-----------+-------------+----------------+------------------+------------------+
|condition| min(id)|min(price)|min(bedrooms)|min(bathrooms)|min(floors)|min(waterfront)|min(view)|min(condition)|min(grade)|min(yr_built)|min(yr_renovated)|min(zipcode)|min(m2_living)|min(m2_lot)|min(m2_above)|min(m2_basement)|  min(m2_living15)|     min(m2_lot15)|
+---------+--------+----------+-------------+--------------+-----------+---------------+---------+--------------+----------+-------------+-----------------+------------+--------------+-----------+-------------+----------------+------------------+------------------+
|        1|40000362|     78000|            0|           0.0|        1.0|              0|        0|             1|         1|         1900|                0|       98004|        26.941|   152.1702|      

*Groepeer de rijen volgens de `waterfront` kolom en aggregeer volgens het gemiddelde.*

In [23]:
data.groupBy("waterfront").mean().show()

+----------+-------------------+------------------+------------------+------------------+------------------+---------------+-------------------+------------------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+-----------------+------------------+------------------+
|waterfront|            avg(id)|        avg(price)|     avg(bedrooms)|    avg(bathrooms)|       avg(floors)|avg(waterfront)|          avg(view)|    avg(condition)|       avg(grade)|     avg(yr_built)|avg(yr_renovated)|     avg(zipcode)|    avg(m2_living)|       avg(m2_lot)|     avg(m2_above)| avg(m2_basement)|  avg(m2_living15)|     avg(m2_lot15)|
+----------+-------------------+------------------+------------------+------------------+------------------+---------------+-------------------+------------------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+-------

Niet alle methoden hebben nood aan een groupby call. In plaats darvan kan je de algemene agg() methode oproepen. 
Dit kan alle rijen in de dataframe aggregeren in een kolom.  
`df.agg({'column_name':'operation'})` de operation kan mean,min,max,count en sum zijn.

*Gebruik de algemene agg() methode om de gemiddelde prijs te vinden*

In [24]:
data.agg({"price": "mean"}).show()

+-----------------+
|       avg(price)|
+-----------------+
|540088.1417665294|
+-----------------+



Deze functie geeft echter een DataFrame terug, wat niet zo handig is bv. als je het gemiddelde nodig hebt om berekeningen uit te voeren. Om enkel het getal terug te krijgen moet je het getal als volgt uit de spark DataFrame halen:  
`.collect()[row_number][column_number]`  
*Zorg ervoor dat de algemene agg() methode het gemiddelde als getal terug geeft.*

In [25]:
data.agg({"price": "mean"}).collect()[0][0]

540088.1417665294

*Groepeer eerst alle rijen volgens `yr_renovated` en sla dit op als de variabele `grouped`. 
Gebruik vervolgens de algemene agg() methode om de maximale prijs te vinden per jaar*

In [26]:
grouped = data.groupBy("yr_renovated").agg({"price": "max"})

In [27]:
grouped.show()

+------------+----------+
|yr_renovated|max(price)|
+------------+----------+
|        1959|    397500|
|        1990|   1646000|
|        1975|    685000|
|        1977|   1598890|
|        2003|   2888000|
|        2007|   2750000|
|        1974|    737500|
|        2015|   1485000|
|        1955|    550000|
|        2006|   2160000|
|        1978|    655000|
|        2013|   2500000|
|        1944|    521000|
|        1956|   1160000|
|        1934|    459950|
|        1988|   1900000|
|        1997|   1580000|
|        1994|   2350000|
|        1968|   1245000|
|        2014|   1755000|
+------------+----------+
only showing top 20 rows



### <span style="color:hotpink"> Samenvattende oefening </span>
*wat is de gemiddelde prijs per bouwjaar van huizen die gerenoveerd zijn na 2005, die minimum 3 slaapkamers hebben, die geviewed zijn en die een grade boven 5 hebben?*


In [28]:
data.filter((data["yr_renovated"] > 2005) & (data["bedrooms"] >= 3) & (data["view"] == 1) & (data["grade"] > 5)).groupBy("yr_built").agg({"price": "mean"}).show()

+--------+----------+
|yr_built|avg(price)|
+--------+----------+
|    1924|  991700.0|
|    1955|  660000.0|
|    1952|  900000.0|
|    1968| 1444000.0|
|    1967| 1080000.0|
|    1953| 1540000.0|
|    1928| 1199000.0|
+--------+----------+



*Leg met je eigen woorden uit wat het is verschil tussen de `agg()` en de `groupBy()` functie?*

# <span style="color:hotpink"> Machine learning with PySpark </span>

Om machine learning te kunnen toepassen met MlLib verwacht spark een format dat 2 kolommen bevat met de namen: "label" en "features".  
De label kolom moet een numerische label bevatten, dit kan een numerische waarde zijn voor regressie of classificatie.  
De feature kolom moet een vector van alle features bevatten.  
In deze sectie zal je een dataframe omzetten naar het verwachte format en zal je de huisprijzen proberen te voorspellen met lineaire regressie.

In [29]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

# <span style="color:hotpink"> Prepping the data </span>

## <span style="color:hotpink"> Prepping the data </span>
Om de data correct voor te bereiden moeten alle features in een kolom onder de vorm van een vector aanwezig zijn.
Dit kan bereikt worden met de functies: `assembler = VectorAssembler(inputCols=[list_of_column_names],outputCol='features')`  
`new_dataframe = assembler.transform(df)`  
*Maak een lijst met alle kolomnamen die gebruikt kunnen worden als features en transformeer ze met de VectorAssembler.*

In [30]:
data.columns

['id',
 'price',
 'bedrooms',
 'bathrooms',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'yr_built',
 'yr_renovated',
 'zipcode',
 'm2_living',
 'm2_lot',
 'm2_above',
 'm2_basement',
 'm2_living15',
 'm2_lot15']

In [72]:
assembler = VectorAssembler(inputCols=['bedrooms',
 'bathrooms',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'yr_built',
 'yr_renovated',
 'm2_living',
 'm2_lot',
 'm2_above',
 'm2_basement',
 'm2_living15',
 'm2_lot15'],
outputCol = "features")

In [73]:
new_data = assembler.transform(data)

## <span style="color:hotpink"> Splitsen in train en test data </span>

Om de data op te delen in train en test data kun je de volgende functie gebruiken:  
`train_data,test_data = df.randomSplit([float1,float2])`  
*Splits nu de data op waarbij 80% van de data train data is.*

In [74]:
train_data, test_data = new_data.randomSplit([0.8, 0.2])

## <span style="color:hotpink"> Runnen algoritme </span>

We zullen in dit labo werken met lineaire regressie. De te gebruiken functies zijn:  
`lr = LinearRegression(labelCol = 'column_name_target', featuresCol = 'column_name_features', regParam = float)`  
`model = lr.fit(train_data)`  
de parameter `regParam` == lambda.

In [75]:
lr = LinearRegression(labelCol = "price", featuresCol = "features", regParam = 0.1)

In [76]:
model = lr.fit(train_data)

In [77]:
results = model.evaluate(test_data)

In [78]:
results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| 37095.743438228965|
|-10858.406218484044|
| -84716.27519575693|
| -89420.07785481587|
| 133561.65012773126|
| -47148.08892013971|
|-11069.025864235125|
|-205019.14883278124|
|-432172.22158306837|
|  174642.1756992489|
| 252554.39445113204|
|-111626.63180756755|
|-201314.12648160942|
| -519683.6505952431|
| -95561.82785819471|
| -49836.17679054104|
| 186176.25101668388|
|  51594.29104369134|
|-151596.89982040506|
|-193710.23481597845|
+-------------------+
only showing top 20 rows



In [80]:
results.r2

0.6448942668204344

*Extra oefening: maak zelf een min-max scaler met de geziene functies en pas deze toe op de dataset. Resulteert dit in een betere RMSE?*

Nope, it does not

In [195]:
columns_backup = data.columns

In [196]:
data_backup = data

In [197]:
def MinMaxScaler(column_name, data):
    new_data = (data[column_name] - data.agg({column_name: "min"}).collect()[0][0]) / (data.agg({column_name: "max"}).collect()[0][0] - data.agg({column_name: "min"}).collect()[0][0])
    return new_data

In [198]:
for column in columns_backup[2:]:
    data_backup = data_backup.withColumn("scaled_" + column, MinMaxScaler(column, data_backup))
    data_backup = data_backup.drop(column)

In [199]:
data_backup.show()

+----------+-------+-------------------+----------------+-------------+-----------------+-----------+----------------+------------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+--------------------+
|        id|  price|    scaled_bedrooms|scaled_bathrooms|scaled_floors|scaled_waterfront|scaled_view|scaled_condition|      scaled_grade|    scaled_yr_built|scaled_yr_renovated|      scaled_zipcode|    scaled_m2_living|       scaled_m2_lot|     scaled_m2_above| scaled_m2_basement| scaled_m2_living15|     scaled_m2_lot15|
+----------+-------+-------------------+----------------+-------------+-----------------+-----------+----------------+------------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+--------------------+
|7129300520| 221900|0.090909090

In [200]:
assembler = VectorAssembler(inputCols=data_backup.columns[2:], outputCol = "features")

In [201]:
new_data = assembler.transform(data_backup)

In [202]:
train_data, test_data = new_data.randomSplit([0.8, 0.2])

In [206]:
lr = LinearRegression(labelCol = "price", featuresCol = "features", regParam = 0.1)

In [207]:
model = lr.fit(train_data)

In [208]:
results = model.evaluate(test_data)

In [209]:
results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| 32902.794918488245|
| -1527.016520125675|
| 117162.35398686957|
|-174597.92151874118|
| -85174.94747993245|
|-113701.55952548946|
| 138195.45636979106|
| -76510.82654485095|
|-48699.073610318475|
|-28219.026139357942|
| -9673.622785771498|
|-315984.26151442947|
| -304744.6526441891|
| -279434.4997689226|
|-120866.93715086358|
| 14984.191792541067|
| 19291.184769256623|
|   131834.833220982|
| -351978.8274748726|
|-202262.21154159016|
+-------------------+
only showing top 20 rows



In [210]:
results.rootMeanSquaredError

207094.03607339706

In [211]:
results.r2

0.6220336236291849